## Examen Práctico
David Armendáriz - A01570813

### Prepocesamiento

In [14]:
# Importar: Librerías 
import pandas as pd
import numpy as np

In [17]:
# Importar: BD Air Quality
df = pd.read_csv("Air_Quality.csv", index_col=0)
df.head()

,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,Message
Unique ID,,,,,,,,,,,
172653,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,203,Bedford Stuyvesant - Crown Heights,Annual Average 2011,12/01/2010,25.30,NaN
172585,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,203,Bedford Stuyvesant - Crown Heights,Annual Average 2009,12/01/2008,26.93,NaN
336637,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,204,East New York,Annual Average 2015,01/01/2015,19.09,NaN
336622,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,103,Fordham - Bronx Pk,Annual Average 2015,01/01/2015,19.76,NaN
172582,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,104,Pelham - Throgs Neck,Annual Average 2009,12/01/2008,22.83,NaN


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16218 entries, 172653 to 325247
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Indicator ID    16218 non-null  int64  
 1   Name            16218 non-null  object 
 2   Measure         16218 non-null  object 
 3   Measure Info    16218 non-null  object 
 4   Geo Type Name   16218 non-null  object 
 5   Geo Join ID     16218 non-null  int64  
 6   Geo Place Name  16218 non-null  object 
 7   Time Period     16218 non-null  object 
 8   Start_Date      16218 non-null  object 
 9   Data Value      16218 non-null  float64
 10  Message         0 non-null      float64
dtypes: float64(2), int64(2), object(7)
memory usage: 1.5+ MB


In [23]:
# BD: Auxiliares | Purgar datos innecesarios
df_aux = df.copy()

# Filtrar: por contaminante (NO2, PM2.5, O3)
values_to_keep_pollutant = ["Nitrogen dioxide (NO2)", "Fine particles (PM 2.5)", "Ozone (O3)"]
df_aux = df_aux[df_aux['Name'].isin(values_to_keep_pollutant)]

# Filtrar: por Geo Type
values_to_keep_location = ["UHF42", "UHF34", "CD"]
df_aux = df_aux[df_aux['Geo Type Name'].isin(values_to_keep_location)]

df_aux = df_aux[["Name", "Data Value", "Geo Join ID", "Geo Type Name"]]
df_aux.head()

,Name,Data Value,Geo Join ID,Geo Type Name
Unique ID,,,,
172653,Nitrogen dioxide (NO2),25.30,203,UHF34
172585,Nitrogen dioxide (NO2),26.93,203,UHF34
336637,Nitrogen dioxide (NO2),19.09,204,UHF34
336622,Nitrogen dioxide (NO2),19.76,103,UHF34
172582,Nitrogen dioxide (NO2),22.83,104,UHF34


In [79]:
df_aux["Name"].value_counts()

Nitrogen dioxide (NO2)     5265
Fine particles (PM 2.5)    5265
Ozone (O3)                 1890
Name: Name, dtype: int64

In [25]:
# Convertir: UHF34 -> UHF 42
df_aux["Geo Join ID"] = df_aux["Geo Join ID"].replace([105106107, 404406, 501503, 306308, 309310, 305307, 503504, 501502],
                                                          [106, 405, 502, 307, 309, 306, 504, 502])
df_aux["Geo Type Name"] = df_aux["Geo Type Name"].replace("UHF34", "UHF42")

In [37]:
# Convertir: Dataframe -> Numpy Array
# NO2
np_NO2_UHF = df_aux[(df_aux["Name"]=="Nitrogen dioxide (NO2)") & (df_aux["Geo Type Name"]=="UHF42")].drop(df_aux.columns[[0,-1]], axis=1).to_numpy()
np_NO2_CD = df_aux[(df_aux["Name"]=="Nitrogen dioxide (NO2)") & (df_aux["Geo Type Name"]=="CD")].drop(df_aux.columns[[0,-1]], axis=1).to_numpy()

# PM2.5
np_PM25_UHF = df_aux[(df_aux["Name"]=="Fine particles (PM 2.5)") & (df_aux["Geo Type Name"]=="UHF42")].drop(df_aux.columns[[0,-1]], axis=1).to_numpy()
np_PM25_CD = df_aux[(df_aux["Name"]=="Fine particles (PM 2.5)") & (df_aux["Geo Type Name"]=="CD")].drop(df_aux.columns[[0,-1]], axis=1).to_numpy()

# O3
np_O3_UHF = df_aux[(df_aux["Name"]=="Ozone (O3)") & (df_aux["Geo Type Name"]=="UHF42")].drop(df_aux.columns[[0,-1]], axis=1).to_numpy()
np_O3_CD = df_aux[(df_aux["Name"]=="Ozone (O3)") & (df_aux["Geo Type Name"]=="CD")].drop(df_aux.columns[[0,-1]], axis=1).to_numpy()


### Mapper

In [40]:
# Importar: Librerías
import kmapper as km
import sklearn
import warnings
warnings.filterwarnings("ignore")

In [66]:
# Función Mapper
def map_pol(data, data_name, proj, ncubes, pover, Kclusters):
    # Inicializar: Mapper
    mapper = km.KeplerMapper(verbose=1)

    # Generar: Proyección sobre distritos
    projected_data = mapper.fit_transform(data, projection=[proj])

    # Generar: Cubierta
    covering = km.Cover(n_cubes=ncubes, perc_overlap=pover)

    # Generar: Mapeo
    graph = mapper.map(projected_data, data, clusterer=sklearn.cluster.KMeans(n_clusters = Kclusters), cover=covering)

    # Mapeo: Color a base de contaminante
    
    mapper.visualize(graph, path_html="Mapeo C " + data_name + ".html",
                 title="Mapeo Color: Contaminante " + data_name,
                 color_values = data[:,0],
                 color_function_name = "Data Value",
                 node_color_function=np.array(['average','std','sum','max','min']))
    
    colorscale = [
        [0.0, "rgb(231, 76, 60)"], # Rojo Bronx
        [0.25, "rgb(142, 68, 173)"], # Morado Brooklyn  
        [0.5, "rgb(52, 152, 219)"], # Azul Manhatan
        [0.75, "rgb(46, 204, 113)"], # Verde Queens
        [1.0, "rgb(247, 220, 111)"] # Amarillo Staten Island
    ]

    # Mapeo: color a base de casos confirmados
    

    mapper.visualize(graph, path_html="Mapeo D " + data_name + ".html",
                 title="Mapeo Color: Distritos " + data_name,
                 color_values = np.around(data[:,1],-2),
                 color_function_name = "Data Value",
                 node_color_function=np.array(['average','std','sum','max','min']),
                 colorscale=colorscale)

##### NO2

In [70]:
map_pol(np_NO2_UHF, "NO2_UHF42", 1, 5, 0.7, 8)

KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: [1]
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (2964, 2)

..Projecting data using: [1]

..Scaling with: MinMaxScaler()

Mapping on data shaped (2964, 2) using lens shaped (2964, 1)

Creating 5 hypercubes.

Created 67 edges and 40 nodes in 0:00:00.752159.
Wrote visualization to: Mapeo C NO2_UHF42.html
Wrote visualization to: Mapeo D NO2_UHF42.html


In [71]:
map_pol(np_NO2_CD, "NO2_CD", 1, 5, 0.7, 8)

KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: [1]
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (2301, 2)

..Projecting data using: [1]

..Scaling with: MinMaxScaler()

Mapping on data shaped (2301, 2) using lens shaped (2301, 1)

Creating 5 hypercubes.

Created 68 edges and 40 nodes in 0:00:00.687513.
Wrote visualization to: Mapeo C NO2_CD.html
Wrote visualization to: Mapeo D NO2_CD.html


##### PM 2.5

In [72]:
map_pol(np_PM25_UHF, "PM2.5_UHF42", 1, 5, 0.7, 8)

KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: [1]
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (2964, 2)

..Projecting data using: [1]

..Scaling with: MinMaxScaler()

Mapping on data shaped (2964, 2) using lens shaped (2964, 1)

Creating 5 hypercubes.

Created 75 edges and 40 nodes in 0:00:00.893043.
Wrote visualization to: Mapeo C PM2.5_UHF42.html
Wrote visualization to: Mapeo D PM2.5_UHF42.html


In [73]:
map_pol(np_PM25_CD, "PM2.5_CD", 1, 5, 0.7, 8)

KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: [1]
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (2301, 2)

..Projecting data using: [1]

..Scaling with: MinMaxScaler()

Mapping on data shaped (2301, 2) using lens shaped (2301, 1)

Creating 5 hypercubes.

Created 66 edges and 40 nodes in 0:00:00.740683.
Wrote visualization to: Mapeo C PM2.5_CD.html
Wrote visualization to: Mapeo D PM2.5_CD.html


##### O3

In [74]:
map_pol(np_O3_UHF, "O3_UHF42", 1, 5, 0.7, 8)

KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: [1]
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (1064, 2)

..Projecting data using: [1]

..Scaling with: MinMaxScaler()

Mapping on data shaped (1064, 2) using lens shaped (1064, 1)

Creating 5 hypercubes.

Created 70 edges and 40 nodes in 0:00:00.645833.
Wrote visualization to: Mapeo C O3_UHF42.html
Wrote visualization to: Mapeo D O3_UHF42.html


In [75]:
map_pol(np_O3_CD, "O3_CD", 1, 5, 0.7, 8)

KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: [1]
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (826, 2)

..Projecting data using: [1]

..Scaling with: MinMaxScaler()

Mapping on data shaped (826, 2) using lens shaped (826, 1)

Creating 5 hypercubes.

Created 67 edges and 40 nodes in 0:00:00.638921.
Wrote visualization to: Mapeo C O3_CD.html
Wrote visualization to: Mapeo D O3_CD.html
